# Reconocimiento de caras de animales
Voy a seguir el flujo que vimos en clase: preparo las imágenes, saco descriptores HOG, juego con un clustering para ver cómo se reparten y termino con un clasificador en un pipeline completo.

## Pasos que me propongo
1. Listar automáticamente todas las carpetas del dataset para usar todas las imágenes disponibles.
2. Pasar las fotos a escala de grises, hacerles `resize` y guardarlas aparte.
3. Guardar un dataset intermedio con joblib (la profe pidió un pickle).
4. Calcular HOG y mirar un clustering con una gráfica sencilla.
5. Montar un pipeline con KernelPCA (KPE), hacer GridSearch con validación cruzada, clasificar y sacar una matriz de confusión basada en la validación cruzada.

In [ ]:
from pathlib import Path
import random

import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from PIL import Image, ImageOps, UnidentifiedImageError

from skimage.feature import hog

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.cluster import KMeans
from sklearn.decomposition import KernelPCA
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import GridSearchCV, StratifiedKFold, cross_val_predict
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

random.seed(42)
np.random.seed(42)

plt.style.use('seaborn-v0_8')
sns.set_theme()

In [ ]:
BASE_DIR = Path('.').resolve()
DATA_DIR = BASE_DIR / 'AnimalFace' / 'Image'
PROCESSED_DIR = BASE_DIR / 'AnimalFace' / 'processed_64_gray'
OUTPUT_DIR = BASE_DIR / 'AnimalFace' / 'outputs'

IMAGE_SIZE = (64, 64)

PROCESSED_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

if not DATA_DIR.exists():
    raise FileNotFoundError(f'No encontré la carpeta de imágenes en {DATA_DIR}')

CATEGORIES = sorted([p.name for p in DATA_DIR.iterdir() if p.is_dir()])
print(f'Trabajo con {len(CATEGORIES)} clases: {CATEGORIES}')

### Funciones de apoyo
Todo lo que reutilizo lo meto en helpers sencillos para no repetir código.

In [ ]:
from typing import List, Tuple


def collect_image_paths(class_name: str) -> List[Path]:
    folder = DATA_DIR / class_name
    supported = []
    for pattern in ('*.jpg', '*.jpeg', '*.png', '*.bmp'):
        supported.extend(folder.glob(pattern))
    return sorted(supported)


def preprocess_image(path: Path, size: Tuple[int, int]) -> np.ndarray:
    try:
        with Image.open(path) as image:
            image = ImageOps.exif_transpose(image)
            image = ImageOps.grayscale(image)
            image = image.resize(size, Image.Resampling.LANCZOS)
            array = np.asarray(image, dtype=np.float32) / 255.0
    except (UnidentifiedImageError, OSError) as exc:
        raise ValueError(f'No pude cargar {path}') from exc
    return array


def save_preprocessed_image(array: np.ndarray, original_path: Path) -> Path:
    label = original_path.parent.name
    save_dir = PROCESSED_DIR / label
    save_dir.mkdir(parents=True, exist_ok=True)
    save_path = save_dir / f"{original_path.stem}_64_gray.png"
    image_to_save = Image.fromarray((array * 255).astype(np.uint8))
    image_to_save.save(save_path)
    return save_path


def build_dataset_descriptor() -> pd.DataFrame:
    rows = []
    skipped = 0
    for label in CATEGORIES:
        paths = collect_image_paths(label)
        for path in paths:
            try:
                processed = preprocess_image(path, IMAGE_SIZE)
            except ValueError:
                print(f"Salté {path.name} porque PIL no la reconoce bien.")
                skipped += 1
                continue
            saved_path = save_preprocessed_image(processed, path)
            rows.append({
                'label': label,
                'original_path': path.as_posix(),
                'processed_path': saved_path.as_posix()
            })
    df = pd.DataFrame(rows)
    df = df.sort_values('label').reset_index(drop=True)
    if skipped:
        print(f"Salté {skipped} imágenes que estaban dañadas o con formato raro.")
    return df


def show_before_after(df: pd.DataFrame, samples: int = 4) -> None:
    subset = df.groupby('label').head(1)
    subset = subset.sample(min(samples, len(subset)), random_state=42)
    fig, axes = plt.subplots(len(subset), 2, figsize=(6, 3 * len(subset)))
    if len(subset) == 1:
        axes = np.array([axes])
    for (idx, row), (ax_orig, ax_proc) in zip(subset.iterrows(), axes):
        orig = Image.open(row['original_path'])
        proc = Image.open(row['processed_path'])
        ax_orig.imshow(orig)
        ax_orig.set_title(f"{row['label']} original")
        ax_orig.axis('off')
        ax_proc.imshow(proc, cmap='gray')
        ax_proc.set_title('Preprocesada 64x64 gris')
        ax_proc.axis('off')
    plt.tight_layout()
    plt.show()


def hog_from_array(image_array: np.ndarray,
                   orientations: int = 9,
                   pixels_per_cell: Tuple[int, int] = (8, 8),
                   cells_per_block: Tuple[int, int] = (2, 2)) -> np.ndarray:
    return hog(
        image_array,
        orientations=orientations,
        pixels_per_cell=pixels_per_cell,
        cells_per_block=cells_per_block,
        block_norm='L2-Hys',
        feature_vector=True
    )


def load_hog_features(df: pd.DataFrame,
                      orientations: int = 9,
                      pixels_per_cell: Tuple[int, int] = (8, 8),
                      cells_per_block: Tuple[int, int] = (2, 2)) -> np.ndarray:
    features = []
    for path in df['processed_path']:
        image = Image.open(path)
        array = np.asarray(image, dtype=np.float32) / 255.0
        features.append(hog_from_array(array, orientations, pixels_per_cell, cells_per_block))
    return np.vstack(features)


class ImageToHog(BaseEstimator, TransformerMixin):
    def __init__(self, image_size: Tuple[int, int] = IMAGE_SIZE,
                 orientations: int = 9,
                 pixels_per_cell: Tuple[int, int] = (8, 8),
                 cells_per_block: Tuple[int, int] = (2, 2)):
        self.image_size = image_size
        self.orientations = orientations
        self.pixels_per_cell = pixels_per_cell
        self.cells_per_block = cells_per_block

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        features = []
        for path in X:
            array = preprocess_image(Path(path), self.image_size)
            features.append(hog_from_array(
                array,
                orientations=self.orientations,
                pixels_per_cell=self.pixels_per_cell,
                cells_per_block=self.cells_per_block
            ))
        return np.vstack(features)

### Preparo el dataset completo
Ahora recorro todas las carpetas y guardo la versión 64x64 en gris para no tener que regenerarla cada vez.

In [ ]:
dataset_df = build_dataset_descriptor()
print(dataset_df.head())
print()
print(dataset_df['label'].value_counts())

show_before_after(dataset_df)

joblib.dump(dataset_df, OUTPUT_DIR / 'dataset_descriptor.joblib')
print(f"Descriptor guardado en {OUTPUT_DIR / 'dataset_descriptor.joblib'}")

### HOG + KPE (KernelPCA) + clustering
Uso KernelPCA como la parte "KPE" que nos comentó la profe y luego miro un KMeans para ver cómo quedan los grupos en las características HOG.

In [ ]:
hog_features = load_hog_features(dataset_df)

joblib.dump({
    'hog_features': hog_features,
    'labels': dataset_df['label'].to_numpy(),
    'paths': dataset_df['processed_path'].to_numpy()
}, OUTPUT_DIR / 'hog_features.joblib')

print(f"HOG guardado en {OUTPUT_DIR / 'hog_features.joblib'}")

kpca_viz = KernelPCA(n_components=2, kernel='rbf', gamma=0.03, random_state=42)
hog_reduced = kpca_viz.fit_transform(hog_features)

kmeans = KMeans(n_clusters=len(CATEGORIES), random_state=42, n_init='auto')
clusters = kmeans.fit_predict(hog_features)

cluster_counts = pd.Series(clusters).value_counts().sort_index()
print('Conteo por cluster:')
print(cluster_counts)

fig, axes = plt.subplots(1, 2, figsize=(12, 5))
scatter = axes[0].scatter(hog_reduced[:, 0], hog_reduced[:, 1], c=clusters, cmap='tab20', s=15)
axes[0].set_title('Clusters sobre HOG reducidos (KPE)')
axes[0].set_xlabel('Comp 1')
axes[0].set_ylabel('Comp 2')
legend1 = axes[0].legend(*scatter.legend_elements(), title='Cluster', loc='best')
axes[0].add_artist(legend1)

axes[1].bar(cluster_counts.index, cluster_counts.values, color='teal')
axes[1].set_xticks(cluster_counts.index)
axes[1].set_xlabel('Cluster')
axes[1].set_ylabel('Número de imágenes')
axes[1].set_title('Cantidad de imágenes por cluster')

plt.tight_layout()
plt.show()

### Pipeline con GridSearch y validación cruzada
Armo el pipeline con: preprocesado → HOG → KernelPCA → escalado → SVM, hago el GridSearch con todo el dataset y validación cruzada y guardo el mejor modelo junto con los resultados.

In [ ]:
X = dataset_df['original_path'].to_numpy()
y = dataset_df['label'].to_numpy()

pipeline = Pipeline([
    ('hog', ImageToHog()),
    ('kpca', KernelPCA(kernel='rbf', n_components=30, gamma=0.03, fit_inverse_transform=False, random_state=42)),
    ('scaler', StandardScaler()),
    ('clf', SVC(probability=False, random_state=42))
])

param_grid = {
    'hog__orientations': [8, 9],
    'hog__pixels_per_cell': [(8, 8), (16, 16)],
    'kpca__n_components': [20, 30, 40],
    'kpca__gamma': [0.01, 0.03],
    'clf__C': [1, 5, 10],
    'clf__gamma': ['scale', 0.01]
}

grid_search = GridSearchCV(
    pipeline,
    param_grid=param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=2
)

grid_search.fit(X, y)

print('Mejor score CV:', grid_search.best_score_)
print('Mejores parámetros:', grid_search.best_params_)

best_model = grid_search.best_estimator_

joblib.dump(best_model, OUTPUT_DIR / 'animal_face_pipeline.joblib')
joblib.dump(grid_search.cv_results_, OUTPUT_DIR / 'grid_search_results.joblib')
print(f"Modelo guardado en {OUTPUT_DIR / 'animal_face_pipeline.joblib'}")
print(f"Resultados CV guardados en {OUTPUT_DIR / 'grid_search_results.joblib'}")

In [ ]:
cv_strategy = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
y_pred = cross_val_predict(best_model, X, y, cv=cv_strategy, n_jobs=-1)

print(classification_report(y, y_pred))

cm = confusion_matrix(y, y_pred, labels=best_model.classes_)
fig, ax = plt.subplots(figsize=(8, 6))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=best_model.classes_)
disp.plot(ax=ax, cmap='Blues', xticks_rotation=45, colorbar=False)
plt.title('Matriz de confusión (validación cruzada)')
plt.tight_layout()
plt.show()

### Cosas para mejorar luego
- Probar con más parámetros del SVM o incluso otros clasificadores para comparar resultados.
- Mirar si se puede reducir el tiempo de GridSearch ajustando mejor la búsqueda o usando menos componentes en KernelPCA.
- Crear un pequeño dashboard para visualizar ejemplos mal clasificados según la validación cruzada.